# 

# Automated Optical Inspection at Variass
Help Variass in predicting the defect rates of assembled products based on their specifications

predict the quality (rate of defects per defect type) based on a given Valor code (or list of Valor codes cq the bill of materials of a completely new product)

## Step 1: Introduction

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Step 1 : Introduction
Read excel AOI, order list and VPL (taking around 31s) and converting it to csv for better performance

In [2]:
file_path_order = "data/initial/Order picklists.xlsx"
file_path_vpl = "data/initial/VPL list.xlsx"
file_path_alternatives = "data/initial/Alternatives.xlsx"
pd.read_excel(file_path_order).to_csv("data/orders.csv", index=False)
pd.read_excel(file_path_vpl).to_csv("data/vpl_codes.csv", index=False)
pd.read_excel(file_path_alternatives).to_csv("data/alternatives.csv", index=False)

Or if the csv files are already there read the new csv files as three new dataframes 

In [47]:
df_aoi_defects = pd.read_csv("data/initial/AOI defects last year.csv")
df_orders = pd.read_csv("data/orders.csv")
df_vpl = pd.read_csv("data/vpl_codes.csv")
df_alternatives = pd.read_csv("data/alternatives.csv")

In [38]:
df_aoi_defects.head()

,overkill,defecttypestring,moduleid,refid,frameid,windowid,isimportantwindow,partnumber,partid,windowtype,...,framerecty4,defectrectx1,defectrectx2,defectrectx3,defectrectx4,defectrecty1,defectrecty2,defectrecty3,defectrecty4,id
0,0,Missing,0,SC68-P,33,2,0,UHW-00003,5532728F-188C-43fc-97D8-C9FBBE46CF3B,1,...,167846.0,499243.0,500321.0,500321.0,499243.0,149534.0,149534.0,150579.0,150579.0,1
1,0,Missing,0,U1-KD,25,0,0,IR9-00013,6DC3F823-57D2-4817-8CC5-53653A2DD0A0,1,...,149048.0,428648.0,436050.0,436050.0,428648.0,114934.0,114934.0,116459.0,116459.0,2
2,0,Missing,0,U1-KD,25,1,0,IR9-00013,6DC3F823-57D2-4817-8CC5-53653A2DD0A0,5,...,149048.0,436675.0,448295.0,448295.0,436675.0,117988.0,117988.0,119084.0,119084.0,3
3,0,Missing,0,U1-KD,25,2,0,IR9-00013,6DC3F823-57D2-4817-8CC5-53653A2DD0A0,5,...,149048.0,424899.0,436519.0,436519.0,424899.0,117991.0,117991.0,119087.0,119087.0,4
4,0,Missing,0,U1-KD,25,3,0,IR9-00013,6DC3F823-57D2-4817-8CC5-53653A2DD0A0,1,...,149048.0,437847.0,440247.0,440247.0,437847.0,120726.0,120726.0,123247.0,123247.0,5


In [39]:
df_orders.head()

,Order,part number,total amount
0,1036933,CAP-00693,504.0
1,1036933,CMO-00180,252.0
2,1036933,CZZ-00214,252.0
3,1036933,CZZ-00215,252.0
4,1036933,CZZ-00216,252.0


In [40]:
df_vpl.head()

,partnumber,mpn,VPLpackage,Material type,Position type,Package type,Lead type,Pitch,Subtype
0,VVH-00021,TSM4ZJ202KR10,PDSO-J3/XX-L48W46T37,P,D,SO,J3,X,X
1,INS-00410,LF353M/NOPB,PDSO-G8/FX-L49W39T18,P,D,SO,G8,F,X
2,VWO-00003,82541300,XDXD-R2/XR-L32W16T15,X,D,XD,R2,X,R
3,FMU-00181,NFM3DCC102R1H3L,XXXD-R3/XL-L32W13T7,X,X,XD,R3,X,L
4,KKT-00218,T495X337K010ATE100,PDSO-C2/XX-L73W43T40,P,D,SO,C2,X,X


In [41]:
df_alternatives.head()

,Original Part,description,Alternative Part,description.1
0,KS55-00005,SMD 330nF 10% 100V X7R 2220 SYF,KKT-00219,SMD 330nF 5% 100V X7R 2220 KEM
1,WBO-00013,SMD 240R 1% 0.25W 1206 100ppm BOU,WVH-00813,SMD 240R 1% 0.25W 1206 100ppm VIS
2,KSI-00076,#SMD 22pF 5% 50V C0G 0603 EPC,KYA-00007,SMD 22pF 5% 50V C0G 0603 YAG
3,KSI-00076,#SMD 22pF 5% 50V C0G 0603 EPC,KYA-00007,SMD 22pF 5% 50V C0G 0603 YAG
4,KSI-00076,#SMD 22pF 5% 50V C0G 0603 EPC,KYA-00007,SMD 22pF 5% 50V C0G 0603 YAG


## First look at the four created dataframes
After creating four different dataframes and looking at the information from the assignment case and lecture slides we can observe an describe the following: 

### df_aoi_defects
- *defecttypestring* = Defecttypestring: type of defect
- *reviewd* = 2 after manual inspection the 3D defect is approved, 1 after manual inspection the 3D defect is disapproved (false call)
- *refid* = reference identification with is the position at the PCBA
- *Partnumber* = link to the orderpick list and VPL list

### df_orders
- *Order* = each order corresponds to an part needed for that order and its total amount
- *part number* = each part number corresponds an specific part needed for that specific assembly order
- *total amount* = total amount of parts needed to fulfil the order 

### df_vpl
- *partnumber* = each partnumber within this dataframe has an corresponding VPL code
- *VPL package* = unique code that describes the assembly used by Variass 

![VPL code meaning](./context/VPL.png)

### df_alternatives
Variass sources parts from different suppliers. The effect of this is that there are sometimes different partnumber names for the same part.
- *Orginal part* = the orginal part used in the assembly process
- *Alternative part* = provided by the supplier to replace the orginal part in which an orginal part can have multiple alternative parts.

With the dataframes ready and the data imported we can go to the next step of data preperation for analysis.

# Step 2 : Data preperation 
**Assignment:** The first step of the analysis is to collect the relevant data from different files and bring the
information together in some form that facilitates your analysis. For instance, it should be
clear that you need to match partnumber’s with VPL codes to aggregate order- and defectrelated data. In preparing your data, consider creating quantitative or numerical attributes
out of ‘coded’ attributes.

Let's first look if that four different dataframes have any missing values. Based on the information of the case and slides we can take a look at specific columns per dataframe

In [42]:
print("df_aoi_defects")
print(df_aoi_defects[["reviewed", "defecttypestring", "refid", "partnumber"]].isnull().sum())
print("\ndf_orders")
print(df_orders[["Order", "part number", "total amount"]].isnull().sum())
print("\ndf_vpl")
print(df_vpl[["partnumber", "VPLpackage"]].isnull().sum())
print("\ndf_alternative")
print(df_alternatives[["Original Part", "Alternative Part"]].isnull().sum())

df_aoi_defects
reviewed               0
defecttypestring    5655
refid                  0
partnumber          4305
dtype: int64

df_orders
Order           0
part number     0
total amount    0
dtype: int64

df_vpl
partnumber    0
VPLpackage    0
dtype: int64

df_alternative
Original Part       0
Alternative Part    0
dtype: int64


We observe that df_aoi_defects has nan values in columns defecttypestring and partnumber, the other dfs don't have nan values. Focussing at the df_aoi_defects specifically starting with the partnumber column the nan values in this column pose an problem, because without an partnumber no link can be made between the other dfs. The missing values on the defecttypestring column are also problematic I assume that in this case an defect has be registered by the AOI machine but no defect type could be linked to it. Lets analyse this further by using the value_counts function.        

In [43]:
df_aoi_defects[df_aoi_defects["defecttypestring"].isnull()][["reviewed", "defecttypestring", "refid", "partnumber"]].head()

,reviewed,defecttypestring,refid,partnumber
655,1,NaN,R106,WVH-00458
684,1,NaN,C452,KAX-00382
808,2,NaN,R406,WYA-00247
927,2,NaN,R410,WYA-00247
951,2,NaN,R430,WYA-00247


As we observe the first 5 rows having an defectypestring of we see true-positives and false positives. This means that the machine defected an defect and this was in some cases accepted by the manual inspection, but propably the type of defect is not registered in the vpl code at Variass. Lets look even further into this. 

In [44]:
df_aoi_defects[df_aoi_defects["defecttypestring"].isnull()].value_counts("reviewed")


reviewed
2    5409
1     246
dtype: int64

As we can see the alot of the rows that cannot be linked to an defecttypestring are reviewed as an true defect (5409). I think that this has to do with that the machine found an new type of defect that isn't yet in registered in the VPL codes, But an defect it is non the less.  

**To conclude**

Having nan values in the partnumber columns in the df_aoi_defects is an problem because we cannot match this with the other dataframes. Therefore, these rows should be excluded from the analysis. For the defecttypestring however its difficult on the one hand you the AOI machine detected I assume these new defects and this should be taken into account in calculating the total defect rate, but on the other hand these rows cannot be used for specific analysis on defect types. For now we will remove the partnumber having nan values and leave the nan valued defecttypestrings in the dataframe.

Next, we will look at the VPL codes in df_vpl.

In [45]:
df_vpl.head()

,partnumber,mpn,VPLpackage,Material type,Position type,Package type,Lead type,Pitch,Subtype
0,VVH-00021,TSM4ZJ202KR10,PDSO-J3/XX-L48W46T37,P,D,SO,J3,X,X
1,INS-00410,LF353M/NOPB,PDSO-G8/FX-L49W39T18,P,D,SO,G8,F,X
2,VWO-00003,82541300,XDXD-R2/XR-L32W16T15,X,D,XD,R2,X,R
3,FMU-00181,NFM3DCC102R1H3L,XXXD-R3/XL-L32W13T7,X,X,XD,R3,X,L
4,KKT-00218,T495X337K010ATE100,PDSO-C2/XX-L73W43T40,P,D,SO,C2,X,X


The df_vpl consists of partnumber, mpn, vpl code, followed by 6 columns containing substrings of the vpl code with all include some information. Let's take the vpl code from the first row and explain last 6 columns.

![VPL code meaning](./context/VPL.png)

VPLpackage of the first row is PDSO-J3/XX-L48W46T37:
- Material type is the P in **P**DSO-J3/XX-L48W46T37 which equals plastic
- Position type is the D in P**D**SO-J3/XX-L48W46T37
- Package type is the SO in PD**SO**-J3/XX-L48W46T37
- Leadtype is the J3 in PDSO-**J3**/XX-L48W46T37 
- Pitch is the X in PDSO-J3/**X**X-L48W46T37
- Subtype is the X in PDSO-J3/X**X**-L48W46T37 
- Length is the L48 in PDSO-J3/XX-**L48**W46T37
- Width is the W46 in PDSO-J3/XX-L48**W46**T37
- Height is the T37 in PDSO-J3/XX-L48W46**T37**

Note that Lead form and lead count in the image are combined in df_vpl and that the Length, Width and Height should ideally have there own columns like Material type, Position type, Package type, Lead type, Pitch and Subtype and the Lead type column should be split into two different columns for lead form and lead count. Let's start with the implementation of these suggestions. 

Starting with the lead type let's look first what type of values the lead type column consists of by using str function to extract the lead form and lead count.  

In [50]:
df_vpl["Lead type"].str[0].value_counts()


R    5921
G    4890
N    1797
C    1092
W     866
F     689
P     662
T     549
L     497
B     277
J      96
X      88
Y       1
Name: Lead type, dtype: int64

The first part of the lead type is an letter depicting the form type of the assembly. Now lets check the let counts (number of lead forms on the assembly).

In [53]:
df_vpl["Lead type"].str[1:].value_counts()

2         8658
8         1550
3         1156
4          951
6          624
          ... 
16(28)       1
5(12)        1
357          1
684          1
94           1
Name: Lead type, Length: 178, dtype: int64

As the first element of the lead type is the form type of the assembly the rest are number or count of these types of lets. Running an value counts function on this subset results in numbers, but also numbers with parentheses around them which is strange. Lets look deeper in these numbers with parentheses 

In [88]:
# source: https://stackoverflow.com/questions/58478230/how-to-return-match-with-a-string-that-contains-parentheses-in-pandas
print("Number of rows in lead type with parentheses in them are", df_vpl["Lead type"].str[1:].str.contains("\)|\(").sum())
df_vpl[df_vpl["Lead type"].str[1:].str.contains("\)|\(")].head()

Number of rows in lead type with parentheses in them are 50


,partnumber,mpn,VPLpackage,Material type,Position type,Package type,Lead type,Pitch,Subtype
385,000000000000421299,09 03 164 6921,PDXC-T66(98)/HH-L940W130T110,P,D,XC,T66(98),H,H
1555,CHX-00024,09 03 164 6921,PDXC-T66(98)/HH-L940W130T110,P,D,XC,T66(98),H,H
1557,CHX-00033,09 03 164 6921,PDXC-T66(98)/HH-L940W130T110,P,D,XC,T66(98),H,H
3368,ILT-00100,LTC1535CSW#PBF,PDSO-G16(28)/FX-L179W75T25,P,D,SO,G16(28),F,X
3727,IHU-00009,HY534256ALJ-60,PDSO-J20(26)/FG-L171W76T36,P,D,SO,J20(26),F,G


As we don't really know why there are 50 vplpackages that have parentheses in them. The first row also has an weird partnumber and mpn of only integers. As this these VPLpackage codes are not valid VPL codes we will remove them from the dataset. 

In [89]:
df_vpl = df_vpl[~df_vpl["Lead type"].str[1:].str.contains("\)|\(")]
print("Number of rows in lead type with parentheses in them are", df_vpl["Lead type"].str[1:].str.contains("\)|\(").sum())

Number of rows in lead type with parentheses in them are 0


Now we can focus on the splitting of the lead type into lead form and lead count. 

In [90]:
df_vpl["Lead form"] = df_vpl["Lead type"].str[0]
df_vpl["Lead count"] = df_vpl["Lead type"].str[1:].astype(int)
df_vpl.drop(columns=["Lead type"])
df_vpl = df_vpl[["partnumber", "mpn", "VPLpackage", "Material type", "Position type", "Package type", "Lead form", "Lead count", "Pitch", "Subtype"]]
df_vpl.head()

,partnumber,mpn,VPLpackage,Material type,Position type,Package type,Lead form,Lead count,Pitch,Subtype
0,VVH-00021,TSM4ZJ202KR10,PDSO-J3/XX-L48W46T37,P,D,SO,J,3,X,X
1,INS-00410,LF353M/NOPB,PDSO-G8/FX-L49W39T18,P,D,SO,G,8,F,X
2,VWO-00003,82541300,XDXD-R2/XR-L32W16T15,X,D,XD,R,2,X,R
3,FMU-00181,NFM3DCC102R1H3L,XXXD-R3/XL-L32W13T7,X,X,XD,R,3,X,L
4,KKT-00218,T495X337K010ATE100,PDSO-C2/XX-L73W43T40,P,D,SO,C,2,X,X


The Leadtype is split between Lead form and Lead count (like in the image), now let's extract the Length, Width and Height out of the VPLpackage columns and assign these values to three columns as well. 

In [91]:
df_vpl.head()

,partnumber,mpn,VPLpackage,Material type,Position type,Package type,Lead form,Lead count,Pitch,Subtype
0,VVH-00021,TSM4ZJ202KR10,PDSO-J3/XX-L48W46T37,P,D,SO,J,3,X,X
1,INS-00410,LF353M/NOPB,PDSO-G8/FX-L49W39T18,P,D,SO,G,8,F,X
2,VWO-00003,82541300,XDXD-R2/XR-L32W16T15,X,D,XD,R,2,X,R
3,FMU-00181,NFM3DCC102R1H3L,XXXD-R3/XL-L32W13T7,X,X,XD,R,3,X,L
4,KKT-00218,T495X337K010ATE100,PDSO-C2/XX-L73W43T40,P,D,SO,C,2,X,X


In [127]:
df_l_w_t = df_vpl["VPLpackage"].str.split("-").str[-1].str.split("L|W|T", n=3, expand=True) 
df_l_w_t.columns = [0, "L", "W", "T"]
df_vpl["L"], df_vpl["W"], df_vpl["T"] = df_l_w_t["L"].astype(int), df_l_w_t["W"].astype(int), df_l_w_t["T"].astype(int)
df_vpl.head()

,partnumber,mpn,VPLpackage,Material type,Position type,Package type,Lead form,Lead count,Pitch,Subtype,L,W,T
0,VVH-00021,TSM4ZJ202KR10,PDSO-J3/XX-L48W46T37,P,D,SO,J,3,X,X,48,46,37
1,INS-00410,LF353M/NOPB,PDSO-G8/FX-L49W39T18,P,D,SO,G,8,F,X,49,39,18
2,VWO-00003,82541300,XDXD-R2/XR-L32W16T15,X,D,XD,R,2,X,R,32,16,15
3,FMU-00181,NFM3DCC102R1H3L,XXXD-R3/XL-L32W13T7,X,X,XD,R,3,X,L,32,13,7
4,KKT-00218,T495X337K010ATE100,PDSO-C2/XX-L73W43T40,P,D,SO,C,2,X,X,73,43,40


The VPLpackage column in df_vpl was splitted based on the dash (-) at first, making the last element the LxxWxxTxx of the VPL code which we need. Next, an second split on this last element based on the characters L or W or T with an max split of 3 times and expanded to an new dataframe called df_l_w_t. These columns are then allocated to the df_vpl as columns L, W and T converted as integers. Now the Length, width and height are extracted from the VPLpackage column we can go further an merge the dataframes.    

## Merging the dataframes
To finish the data preperation part we need to merge the different dataframes.

In [130]:
df_merged_order_vpl = pd.merge(df_orders, df_vpl[["partnumber", "VPLpackage"]], left_on="part number", right_on="partnumber")
df_merged_vpl_and_defects = pd.merge(df_vpl[["partnumber", "VPLpackage", "Material type", "Position type", "Package type", "Lead form", "Lead count", "Pitch", "Subtype", "L", "W", "T"]], df_aoi_defects[["reviewed", "defecttypestring", "refid", "partnumber"]], left_on="partnumber", right_on="partnumber")

In [131]:
df_merged_order_vpl.head()

,Order,part number,total amount,partnumber,VPLpackage
0,1036933,CAP-00693,504.0,CAP-00693,PZXC-L12/HH-L198W52T53
1,1038499,CAP-00693,200.0,CAP-00693,PZXC-L12/HH-L198W52T53
2,1038502,CAP-00693,200.0,CAP-00693,PZXC-L12/HH-L198W52T53
3,1038699,CAP-00693,200.0,CAP-00693,PZXC-L12/HH-L198W52T53
4,1038701,CAP-00693,200.0,CAP-00693,PZXC-L12/HH-L198W52T53
...,...,...,...,...,...
59715,1042104,IAD-00370,750.0,IAD-00370,PDSO-N11/MX-L30W30T8
59716,1042217,IAD-00370,1725.0,IAD-00370,PDSO-N11/MX-L30W30T8
59717,1042218,IAD-00370,1200.0,IAD-00370,PDSO-N11/MX-L30W30T8
59718,1042219,IAD-00370,1800.0,IAD-00370,PDSO-N11/MX-L30W30T8


In [132]:
df_merged_vpl_and_defects.head()

,partnumber,VPLpackage,Material type,Position type,Package type,Lead form,Lead count,Pitch,Subtype,L,W,T,reviewed,defecttypestring,refid
0,VVH-00021,PDSO-J3/XX-L48W46T37,P,D,SO,J,3,X,X,48,46,37,1,Missing,R6
1,VVH-00021,PDSO-J3/XX-L48W46T37,P,D,SO,J,3,X,X,48,46,37,1,Missing,R6
2,VVH-00021,PDSO-J3/XX-L48W46T37,P,D,SO,J,3,X,X,48,46,37,1,Missing,R6
3,VVH-00021,PDSO-J3/XX-L48W46T37,P,D,SO,J,3,X,X,48,46,37,1,Missing,R6
4,VVH-00021,PDSO-J3/XX-L48W46T37,P,D,SO,J,3,X,X,48,46,37,1,Missing,R6


We use the pd.merge in order to inner merge the orders and vpl codes and the vpl codes and defects both on partnumber. By default the pd.merge uses an inner merge meaning that unmatched rows will be discarded. This is what we need because an order without an vpl code is not of intressed to us and an vpl code without an defect is not intressing for the further analysis.  

As the df_aoi_defect and df_vpl are inner merged on partnumber, we solve the nan value issue on the partnumber column in df_aoi_defect described earlier. These nan partnumber cells won't match any of the partnumber cells in the df_vpl and are discarded. The nan valued defecttypestring can still be found in the new df_merged_vpl_and_defects dataframe, let's test this.  

In [27]:
df_merged_vpl_and_defects["defecttypestring"].isnull().sum()

5375

See, the nan defecttypestrings are still in there and will be used in calculating the total defect rate in step 3. Besides this we can observe that the Lead type is replaced by Lead form and Lead count and the Length, Width and Height of part have there own columns. Hence, we are ready to proceed to the next part Defect rates.

# Step 3 : Defect rates
In  this  step,  you  will  make  descriptive  estimations  on  the  defect  rates  without  considering 
any product specifications.
- (a)  Find the defect rate returned by the automated inspection system on the overall and per defect type.  
- (b)  Find the true- and false-positive rates overall and per defect type, based on the information on both automated and manual inspections

The defects found by the 3D machine are based on the column 'reviewed'. Starting with this column lets use value_counts to check how many times 2 (approved after inspection) and 1 (disapproved after inspection) occure in the dataset. 

In [133]:
df_merged_vpl_and_defects.value_counts(subset=["reviewed"])

reviewed
2           680553
1           226263
0             6134
dtype: int64

We see something intressting in the result of value counts on the review column, as can be observerd in the result the review column also includes 0 values, it is assumeed that this 0 value means that the manual inspection didn´t reviewed these parts yet. Focussing on the calculation of the defect rate overall, the length merged dataframe of vpl codes and defects is divided by the sum of total amount column in df_orders and multiplied by 100. 

In [134]:
total_amount_parts_ordered = df_orders["total amount"].sum()
print("Total amount of orders", total_amount_parts_ordered)
# defect rate by the automated inspection system on the overall, but 0 reviews should be included 
defect_rate_overall = (len(df_merged_vpl_and_defects) / total_amount_parts_ordered) * 100
# get all defects 
print("Defect rate returned by the automated inspection system overall is", round(defect_rate_overall, 3), "%")

Total amount of orders 76993396.73099999
Defect rate returned by the automated inspection system overall is 1.186 %


We observer based on number of defects and the total order amount that the automated inspection system finds out of 76993396.73 parts ordered that 1.2% are found of being defect. Next, the value_counts method is used on the column defect types in order to count the number of occurances per defect type. The results of the value_counts function is then placed in an dataframe called df_defects. Based on the new dataframe df_defects the defect rate per defect type is calculated by dividing the type of defect counts by total amount of parts ordered and multiplied by 100. 

In [135]:
defect_types = df_merged_vpl_and_defects.value_counts(subset=["defecttypestring"])
# source: https://stackoverflow.com/questions/47136436/python-pandas-convert-value-counts-output-to-dataframe
df_defects = pd.DataFrame(defect_types)
df_defects = df_defects.reset_index()
df_defects.columns = ["defect_type", "amount"]
# calculate the defect rate per defect type
df_defects["defect_rate"] = (df_defects["amount"] / total_amount_parts_ordered) * 100
df_defects

,defect_type,amount,defect_rate
0,InsufficientSolder,389902,0.506410
1,Missing,200316,0.260173
2,WrongPart,104351,0.135532
3,LiftedLead,55738,0.072393
4,Bridge,32256,0.041895
5,LiftedPackage,25909,0.033651
6,Polarity,22578,0.029325
7,Shift,21300,0.027665
8,Tomstone,15710,0.020404
9,NoSolder,11843,0.015382


Based on the AOI defect type excel file there are 37 different defect types of which 25 are observed here, why 25? Because on index numbers 10 and 13 of the value counts result above we observe two Dutch words: Tekort and tekort meaning too small, but based on the AOI defect type excel file there is no defect type with that name (or the english translation). Besides this, based on the AOI defect types excel file the 'most used (in green)' defects are Missing, Shift, InsufficientSolder and LiftedPackage, but in the analysis of the dataframe we also observe that wrongpart placement is also quite often found by the AOI machine. Next, we look at the true- and false-positive rates.   

As stated in the powerpoint presentation of Variass slide 14 the AOI machine reviews every part and when the an defect is observed the manual inspection reviews if the defect is correctly observed by the machine, if the machine is correct the manual inspection approves (reviewed = 2 = true-positive) the defect, if the machine is incorrect the manual inspection disapproves (reviewed = 1 = false-positive) the defect. Let's start by finding the true- and false positive rates overall. As found above the reviewed column has some zeros it is assumed that the 0 means that the manual inspection didnt review these parts yet. Therefore, we will leave these out in the following block of code.  

Let's start by finding the true- and false-positive rates overall. 

In [136]:
# get the amount of reviewed 1 and 2 exluding the 0
total_true_false_positives = len(df_merged_vpl_and_defects[df_merged_vpl_and_defects["reviewed"] != 0])
false_postives = len(df_merged_vpl_and_defects[df_merged_vpl_and_defects["reviewed"] == 1])
true_positives = len(df_merged_vpl_and_defects[df_merged_vpl_and_defects["reviewed"] == 2])
assert false_postives + true_positives == total_true_false_positives
false_positive_rate = (false_postives / total_true_false_positives) * 100
true_positive_rate = (true_positives / total_true_false_positives) * 100
print("False positive rate overall (reviewed = 1 = inspection disapproves):", round(false_positive_rate, 3), "%")
print("True positive rate overall (reviewed = 2 = inspection approves):", round(true_positive_rate, 3), "%")
print("Correct defects found by the machine", (total_amount_parts_ordered / 100) * true_positive_rate, "parts")
print("Incorrect defects found by the machine", (total_amount_parts_ordered / 100) * false_positive_rate, "parts")

False positive rate overall (reviewed = 1 = inspection disapproves): 24.951 %
True positive rate overall (reviewed = 2 = inspection approves): 75.049 %
Correct defects found by the machine 57782490.74285438 parts
Incorrect defects found by the machine 19210905.98814561 parts


The true- and false-positive rates of the AOI of 74% and 25% respectivly are calculated by picking the amount of true- and false-postives divided by the sum of true- and false-postives times 100. At first hand the AOI machine with 74% based on 25 defect types is quite on point with its analysis. But based on the historical order data that would mean that 25% of the ordered parts are incorrectly assessed by the machine which is quite alot false inspections over the years. So in the end this quite a dubble story with negatives and postives. Let's look further at the true- and false postives per defect type.  

In [137]:
false_positives_counts = df_merged_vpl_and_defects[df_merged_vpl_and_defects["reviewed"] == 1].value_counts(subset=["defecttypestring"])
df_false_positives = pd.DataFrame(false_positives_counts)
df_false_positives = df_false_positives.reset_index()
df_false_positives.columns = ["defect_type", "false_positives"]
df_false_positives["false_positive_rate"] = (df_false_positives["false_positives"] / false_postives) * 100 
print("Number of defect types having false positive reviews are", len(df_false_positives))
df_false_positives

Number of defect types having false positive reviews are 26


,defect_type,false_positives,false_positive_rate
0,Missing,64586,28.544658
1,InsufficientSolder,32356,14.300173
2,WrongPart,24869,10.991192
3,LiftedPackage,24188,10.690214
4,Shift,19561,8.645249
5,NoSolder,10152,4.486814
6,LiftedLead,8370,3.699235
7,Tekort,7915,3.498142
8,Tomstone,7886,3.485325
9,Bridge,7724,3.413727


Starting with the false positive rate per defect type it can be observed that out of 37 defect types in the AOI defect types excel file, 26 have been manually false positively reviewed by the inspection. If we look at the df_defect number of defect types with is 27 the number of defect types having a false positive review are 26. This means that 1 defect type in df_defects doesn't have an false posistive review. Let's outer merge the df_defects with df_false_positives on defect_type in order to find out which defect type it is. We use outer merge because when the missing defect_type will not have a match between the two dataframes the cell values will be filled with nan values.    

In [138]:
df_defects = pd.merge(df_defects, df_false_positives, on="defect_type", how="outer")
df_defects

,defect_type,amount,defect_rate,false_positives,false_positive_rate
0,InsufficientSolder,389902,0.506410,32356.0,14.300173
1,Missing,200316,0.260173,64586.0,28.544658
2,WrongPart,104351,0.135532,24869.0,10.991192
3,LiftedLead,55738,0.072393,8370.0,3.699235
4,Bridge,32256,0.041895,7724.0,3.413727
5,LiftedPackage,25909,0.033651,24188.0,10.690214
6,Polarity,22578,0.029325,5552.0,2.453782
7,Shift,21300,0.027665,19561.0,8.645249
8,Tomstone,15710,0.020404,7886.0,3.485325
9,NoSolder,11843,0.015382,10152.0,4.486814


After the merge we can indeed see that defect_type LiftedSolder doesnt have any false_positive reviews and is filled with nan values. Next, we need to do the same for the true positives. 

In [140]:
true_positives_counts = df_merged_vpl_and_defects[df_merged_vpl_and_defects["reviewed"] == 2].value_counts(subset=["defecttypestring"])
df_true_positives = pd.DataFrame(true_positives_counts)
df_true_positives = df_true_positives.reset_index()
df_true_positives.columns = ["defect_type", "true_positives"]
df_true_positives["true_positive_rate"] = (df_true_positives["true_positives"] / true_positives) * 100 
print("Number of defect types having true positive reviews are", len(df_true_positives))
df_true_positives

Number of defect types having true positive reviews are 26


,defect_type,true_positives,true_positive_rate
0,InsufficientSolder,355059,52.172131
1,Missing,133569,19.626539
2,WrongPart,78688,11.562362
3,LiftedLead,47249,6.942736
4,Bridge,24466,3.595018
5,Polarity,16763,2.463144
6,Tomstone,7730,1.135841
7,Tilt,2930,0.430532
8,Shift,1726,0.253617
9,LiftedPackage,1715,0.252001


Same as the false positives, the true positives are found in 26 out of 27 defect types. The most true positive reviews found are defect_types insufficientsolder, missing part and wrong part. What is intressing is to see that insufficientsolder has an very high true positive rate (>52%) which means that from all true positive reviews upmost 50% consist of insufficient solder. So insufficient solder is quite a big problem for Variass in there production / assembly line, but on the other hand it also a good thing because in >52% cases the machine detects correct defects of having insufficient solder. Next, let's outer merge the true positives with the df_defects in order finalize the descriptive estimations on the defect rates.

In [141]:
df_defects = pd.merge(df_defects, df_true_positives, on="defect_type", how="outer")
df_defects.fillna(0)

,defect_type,amount,defect_rate,false_positives,false_positive_rate,true_positives,true_positive_rate
0,InsufficientSolder,389902,0.506410,32356.0,14.300173,355059.0,52.172131
1,Missing,200316,0.260173,64586.0,28.544658,133569.0,19.626539
2,WrongPart,104351,0.135532,24869.0,10.991192,78688.0,11.562362
3,LiftedLead,55738,0.072393,8370.0,3.699235,47249.0,6.942736
4,Bridge,32256,0.041895,7724.0,3.413727,24466.0,3.595018
5,LiftedPackage,25909,0.033651,24188.0,10.690214,1715.0,0.252001
6,Polarity,22578,0.029325,5552.0,2.453782,16763.0,2.463144
7,Shift,21300,0.027665,19561.0,8.645249,1726.0,0.253617
8,Tomstone,15710,0.020404,7886.0,3.485325,7730.0,1.135841
9,NoSolder,11843,0.015382,10152.0,4.486814,1678.0,0.246564


# Step 4: Prediction taks and Performance reporting
Next, we will scrutinize the available to make predictions on defect rates considering product specifications by proposing, implementing and showcasing a prediction model. 

Steps for this part include:
<ol type="a">
 <li>Develop  a  single  model  that  predicts  the  defect  rate  per  defect  type,  based  on  the 
inspection data and product specifications captured in the VPL codes. </li>
 <li>Run  your  model  using  all  inspection  data  for  training  and  learning.  Present  your  results 
and . Provide a critical analysis of your findings.</li>
 <li>Run  your  model  using  80%  of  the  inspection  data  for  training  and  testing  and  20%  for 
validation. Present your results.  Provide a critical analysis of your findings.</li>
</ol>

In [142]:
df_merged_order_vpl.head()

,Order,part number,total amount,partnumber,VPLpackage
0,1036933,CAP-00693,504.0,CAP-00693,PZXC-L12/HH-L198W52T53
1,1038499,CAP-00693,200.0,CAP-00693,PZXC-L12/HH-L198W52T53
2,1038502,CAP-00693,200.0,CAP-00693,PZXC-L12/HH-L198W52T53
3,1038699,CAP-00693,200.0,CAP-00693,PZXC-L12/HH-L198W52T53
4,1038701,CAP-00693,200.0,CAP-00693,PZXC-L12/HH-L198W52T53


In [143]:
df_merged_vpl_and_defects.head()

,partnumber,VPLpackage,Material type,Position type,Package type,Lead form,Lead count,Pitch,Subtype,L,W,T,reviewed,defecttypestring,refid
0,VVH-00021,PDSO-J3/XX-L48W46T37,P,D,SO,J,3,X,X,48,46,37,1,Missing,R6
1,VVH-00021,PDSO-J3/XX-L48W46T37,P,D,SO,J,3,X,X,48,46,37,1,Missing,R6
2,VVH-00021,PDSO-J3/XX-L48W46T37,P,D,SO,J,3,X,X,48,46,37,1,Missing,R6
3,VVH-00021,PDSO-J3/XX-L48W46T37,P,D,SO,J,3,X,X,48,46,37,1,Missing,R6
4,VVH-00021,PDSO-J3/XX-L48W46T37,P,D,SO,J,3,X,X,48,46,37,1,Missing,R6


In [146]:
df = df_merged_vpl_and_defects[["defecttypestring",  "VPLpackage", "Material type", "Position type", "Package type", "Lead form", "Lead count", "Pitch", "Subtype", "L", "W", "T"]].value_counts().reset_index()
df = df.rename(columns={0:"vpl_counts"})

df_merged_order_vpl_grouped = df_merged_order_vpl.groupby(by="VPLpackage").sum()
df_merged_order_vpl_grouped = df_merged_order_vpl_grouped.reset_index()

df = pd.merge(df, df_merged_order_vpl_grouped[["VPLpackage", "total amount"]], on="VPLpackage")

df["defect_rate_ppm"] = (df["vpl_counts"] / df["total amount"]) * 1000000
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)


df = df[["VPLpackage", "Material type", "Position type", "Package type", "Lead form", "Lead count", "Pitch", "Subtype", "L", "W", "T", "defecttypestring", "vpl_counts", "total amount", "defect_rate_ppm"]]

df_dummies = pd.get_dummies(df["defecttypestring"])
df = df.join(df_dummies)
df = df.drop(["VPLpackage", "defecttypestring"], axis="columns")
df

C:\Users\tobia\AppData\Local\Temp\ipykernel_18544\2716584624.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_merged_order_vpl_grouped = df_merged_order_vpl.groupby(by="VPLpackage").sum()


,Material type,Position type,Package type,Lead form,Lead count,Pitch,Subtype,L,W,T,...,Polarity,Rotate,Shift,SolderBall,Tekort,Tilt,Tomstone,TurnOver,WrongPart,tekort
0,C,D,XD,R,2,X,R,16,9,5,...,0,0,0,0,0,0,0,0,0,0
1,C,D,XD,R,2,X,R,16,9,5,...,0,0,0,0,0,0,0,0,0,0
2,C,D,XD,R,2,X,R,16,9,5,...,0,0,0,0,0,0,1,0,0,0
3,C,D,XD,R,2,X,R,16,9,5,...,0,0,0,0,0,0,0,0,1,0
4,C,D,XD,R,2,X,R,16,9,5,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6730,P,D,SO,N,7,F,X,49,59,10,...,0,0,0,0,0,0,0,0,0,0
6731,P,D,SO,N,7,F,X,49,59,10,...,0,0,0,0,0,0,0,0,0,0
6732,M,D,SO,N,4,Q,X,32,25,7,...,0,0,0,0,0,0,0,0,0,0
6733,P,D,SO,F,2,X,X,117,50,45,...,0,0,0,0,0,0,0,0,0,0


In [93]:
df_dummies = pd.get_dummies(df[["defecttypestring", "Material type", "Position type", "Package type", "Lead type", "Pitch", "Subtype", "L", "W", "T"]])
# df_dummies["defect_rate"] = df["defect_rate"]
# df_dummies["defect_type"] = df["defecttypestring"]
# df_dummies.to_csv("data/janjan&tobi.csv", index=False)
df_dummies

,L,W,T,defecttypestring_Bridge,defecttypestring_ChipFlying,defecttypestring_ColdSolder,defecttypestring_Damage,defecttypestring_DoubleChip,defecttypestring_ExcessSolder,defecttypestring_ForeignMaterial,...,Subtype_M,Subtype_N,Subtype_O,Subtype_P,Subtype_R,Subtype_S,Subtype_T,Subtype_U,Subtype_X,Subtype_Y
0,16,9,5,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,16,9,5,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,16,9,5,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,16,9,5,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,16,9,5,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6749,190,190,20,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6750,300,102,29,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6751,28,44,21,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6752,38,22,23,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
